In [1]:
import numpy as np
import pandas as pd
import ipywidgets as widgets

from ipywidgets import interact, interactive, fixed, interact_manual
from utilities.visualisation import *

ModuleNotFoundError: No module named 'rosbag_pandas'

In [2]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [3]:
%%html
<style>

div.output_scroll {
    height : auto;
}
</style>

In [4]:
trs = [0.2, 0.3, 0.4, 0.5]
ts = [100,150]

# Select threshold
## Ramps

In [6]:
out_dir = '/data/iakram/Master-Thesis/papers/Thesis/img/select-tr'
from jupyterthemes import jtplot
jtplot.reset()

plt.style.use(['science', 'ieee'])
# get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

ModuleNotFoundError: No module named 'jupyterthemes'

In [7]:
??DataFrameVisualization

In [8]:
def vis_int(t):
    vis = DataFrameVisualization.from_path('/media/francesco/saetta/krock-dataset/slope/csvs/test.csv', time_window=t)
    fig = vis.plot_advancement_box()
    plt.show()
#     plt.gcf().savefig(out_dir + '/slope-box{}.png'.format(t))
interact(vis_int, t=ts)

<function __main__.vis_int>

## Bumps

In [ ]:
def vis_int(t):
    vis = DataFrameVisualization.from_path('/media/francesco/saetta/krock-dataset/tr/csvs/test.csv', time_window=t)
    fig = vis.plot_advancement_box()
    plt.gcf().savefig(out_dir + '/bumps-box{}.png'.format(t))

interact(vis_int, t=ts)

# Datasets
## Statistics

In [ ]:
visualizations = {
    'train': DataFrameVisualization.from_root('/media/francesco/saetta/krock-dataset/train/csvs/'),
    'val': DataFrameVisualization.from_root('/media/francesco/saetta/krock-dataset/val/csvs/'),
#     'test': DataFrameVisualization.from_root('/media/francesco/saetta/krock-dataset/test/csvs/'),
    'new-test-random': DataFrameVisualization.from_root('/media/francesco/saetta/krock-dataset/new-test-random/csvs/'),
   'train-with-rock': DataFrameVisualization.from_root('/media/francesco/saetta/krock-dataset/train-with-rocks'),
#     'new-val': DataFrameVisualization.from_root('/media/francesco/saetta/krock-dataset/new-val/csvs/'),



}

In [ ]:
def vis_dataset(name, tr, t):
    vis = visualizations[name]
    vis.add_advancement(t)
#     vis.df = vis.df[vis.df['advancement'] >= -0.35]
    vis.df = vis.df[vis.df['advancement'] < 0.75]
    fig = vis.show_classes(tr)
    plt.show()
interact(vis_dataset, name=visualizations.keys(), tr=trs, t=ts)

## Different maps
We can explore the advancament on different training maps

### Mean advancement for each map

In [41]:
%matplotlib notebook
from jupyterthemes import jtplot
jtplot.reset()

In [42]:
# merge all the dataframes in each map
clusters = meta2clusters('/media/francesco/saetta/krock-dataset/train/meta.csv', '/media/francesco/saetta/krock-dataset/train/csvs_patches/')

In [51]:
adv_for_each_map = { k: df['advancement'] for k, df in clusters.items()}

In [52]:
adv_for_each_map = sorted(adv_for_each_map.items(), key=lambda kv: kv[1].mean(), reverse=False)

import collections

adv_for_each_map = collections.OrderedDict(adv_for_each_map)

In [55]:
mean_adv_df = pd.DataFrame(data = adv_for_each_map)
mean_adv_df.dropna()
mean_adv_df = mean_adv_df[mean_adv_df >= -0.71]
mean_adv_df=  mean_adv_df[mean_adv_df < 0.71]

# mean_adv_df.plot.bar(x='maps', y='means')
fig = plt.figure(dpi=800, figsize=(16,8))
_ = mean_adv_df.boxplot(rot=90)
fig.savefig('/home/francesco/Documents/Master-Thesis/papers/Thesis/img/datasets/box_for_each_map.png', bbox_inches='tight', transparent=True)
# plt.show()


In [ ]:
list(mean_adv_df.columns)

In [ ]:
def vis_clusters(name, tr, t):
    vis = DataFrameVisualization(clusters[name], time_window=t)
    vis.df = vis.df[vis.df['advancement'] >= 0]
    vis.df = vis.df[vis.df['advancement'] < 1]

    vis.plot_advancement_box()
    vis.show_classes(tr)

interact(vis_clusters, name=clusters.keys(), tr=trs, t=ts)

In [ ]:
def vis_dataset(name, tr, t):
    vis = DataFrameVisualization.from_root('/media/francesco/saetta/krock-dataset/{}/csvs/'.format(name), time_window=t)
    vis.df = vis.df[vis.df['advancement'] >= 0]
    vis.df = vis.df[vis.df['advancement'] < 1]

    vis.plot_advancement_box()
    vis.show_classes(tr)

interact(vis_dataset, name=visualizations.keys(), tr=trs, t=ts)

# Maximum advancement on flat ground

In [ ]:
def vis_dataset(t):
    vis = DataFrameVisualization.from_root('/media/francesco/saetta/flat/csvs_patches/',  time_window=t)
    print(max(vis.df['advancement']))
    vis.plot_advancement_box()


interact(vis_dataset, t=ts)

# Temp
use to visualize on the fly some dataset

In [15]:
from jupyterthemes import jtplot
jtplot.reset()
#
plt.style.use(['science', 'ieee'])
# matplotlib.rcParams['figure.figsize'] = (12,12)
get_ipython().config.get('IPKernelApp', {})['parent_appname'] = ""

In [18]:
from utilities.postprocessing.handlers.functional import read_image
dfs = glob.glob('/media/francesco/saetta/krock-dataset/train/100/csvs/bars1*.csv')
hm = read_image('./maps/train/bars1.png') / 255
patches_dir = '/media/francesco/saetta/krock-dataset/train/100/patches/'

In [24]:
out_dir = '/home/francesco/Desktop/bars1-example-patches'
from utilities.patches import Patch

def store_patches_from_df(df, how_many=10):
    skip = len(df) // how_many
    for idx, row in df[::skip].iterrows():
        fig = plt.figure()
        p = Patch.from_path(patches_dir + '/' + row['images'])
        p.plot2d().savefig('{}/{}-{}.png'.format(out_dir, idx, row['advancement']))

In [28]:
def vis_df(df):
    print(df)
    vis = PatchesAndDataframeVisualization.from_path(df, hm)
#     store_patches_from_df(vis.df)
#     vis.plot_advancement()
    vis.show_traces([vis.df])
    plt.show()

interact(vis_df, df=dfs)

interactive(children=(Dropdown(description='df', options=('/media/francesco/saetta/krock-dataset/train/100/csv…

<function __main__.vis_df(df)>

In [ ]:
vis = DataFrameVisualization.from_root('/media/francesco/saetta/krock-dataset/new-val/csvs', time_window=100)
# vis(0.3)
vis.df['label'] = vis.df['advancement'] > 0.3
vis.df = vis.df[vis.df['label'] == False]

vis(0.3)

In [ ]:

vis = PatchesAndDataframeVisualization.from_df_path_and_hm_path('/media/francesco/saetta/krock-dataset/new-val/csvs/arc_rocks-1-1558626838.6769319.csv', 
                                                 './maps/val/arc_rocks.png',
                                                 max_advancement=0.71,
                                                 time_window=100)
vis.show_traces([vis.df])
vis(0.2)

In [ ]:
    vis = DataFrameVisualization.from_root('/media/francesco/saetta/krock-dataset/new-val/csvs/', time_window=100)
    vis(0.2)


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
from utilities.postprocessing.handlers.functional import read_image

In [ ]:
fig = plt.figure()
hm = read_image('/media/francesco/saetta/krock-dataset/new-test-random/patches/0.71/querry-big-10-10-1558293308.877082-13.png')
sns.heatmap(hm)

In [ ]:
from utilities.patches import *
import cv2 

fig = plt.figure()
hm = read_image('/media/francesco/saetta/krock-dataset/train/patches/0.71/bumps0-2-1558007105.485-277.png')
sns.heatmap(hm)

In [ ]:
fig = plt.figure()
hm = read_image('/media/francesco/saetta/krock-dataset/train/patches/0.71/bars1-1-1558017877.997997-92.png')
# hm = cv2.resize(hm, (8,8))
print(hm.shape)
# hm = cv2.resize(hm, (76,76))
sns.heatmap(hm)